# 5 prepare remote deploy

## prepare turtlebot3
You can use either turtlebot3_simulator (A.) or actual turtlebot3 robot (B.).

### A. use "turtlebot3 simulator" on Ubuntu desktop

#### prepare Ubuntu desktop as a turlebot3 simulator 

1. prepare Ubuntu 16.04 desktop
    * If you use a virtual machine on VirtualBox as Ubuntu desktop, confirm below:
        * "3d acceralation" of display is "OFF"
        * set `export LIBGL_ALWAYS_SOFTWARE=1` to your .bashrc
1. set `export TURTLEBOT3_MODEL=waffle` to your .bashrc
1. install `ros-kinetic-desktop-full` and `ros-kinetic-rqt-*` using `apt`.
1. create ROS workspace.
1. clone repositories of turtlebot3_simulator from github.
    * https://github.com/ROBOTIS-GIT/turtlebot3.git
    * https://github.com/ROBOTIS-GIT/turtlebot3_msgs.git
    * https://github.com/ROBOTIS-GIT/turtlebot3_simulations.git
1. make repositories using `catkin_make`.

### B. use actual "turtlebot3" robot

#### prepare Ubuntu desktop as a turlebot3 simulator

1. prepare turtlebot3
1. make repositories using `catkin_make`.

## change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/container-centric-fiware-demonstration"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/container-centric-fiware-demonstration
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/azure_aks/env

In [ ]:
export TURTLEBOT3_USER=turtlebot3
export TURTLEBOT3_HOST=turtlebot3.local

## setup cert file to connect MQTT broker of fiware

### prepare a command to scp cafile to Ubuntu Desktop and/or turtlebot3

In [ ]:
echo "scp ${PJ_ROOT}/secrets/ca.crt ${TURTLEBOT3_USER}@${TURTLEBOT3_HOST}:/tmp/ca.crt"

### set up cert file

1. create a directory

```bash
turtlebot3@turtlebot3:~$ sudo mkdir /etc/mqtt-kube-operator
```
2. scp the `ca.crt` from local to Ubuntu desktop
3. move cert file to the created directory

```bash
turtlebot3@turtlebot3:~$ sudo mv /tmp/ca.crt /etc/mqtt-kube-operator/
```

## setup docker

1. install required packages

```bash
turtlebot3@turtlebot3:~$ sudo apt update
turtlebot3@turtlebot3:~$ sudo apt update -y
turtlebot3@turtlebot3:~$ sudo apt install apt-transport-https ca-certificates curl software-properties-common
  ```
2. add Docker’s official GPG key:

```bash
turtlebot3@turtlebot3:~$ curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
```
3. set up the stable repository of Docker

```bash
turtlebot3@turtlebot3:~$ sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable"
```
4. install Docker

```bash
turtlebot3@turtlebot3:~$ sudo apt update
turtlebot3@turtlebot3:~$ sudo apt install docker-ce -y
```
5. verify instration

```bash
turtlebot3@turtlebot3:~$ sudo docker run hello-world
```

## setup minikube

1. setup minikube

```bash
turtlebot3@turtlebot3:~$ curl -Lo minikube https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64 && chmod +x minikube && sudo cp minikube /usr/local/bin/ && rm minikube
```
2. verify instration

```bash
turtlebot3@turtlebot3:~$ minikube version
```

## setup kubectl

1. setup kubectl

```bash
turtlebot3@turtlebot3:~$ curl -Lo kubectl https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl && chmod +x kubectl && sudo cp kubectl /usr/local/bin/ && rm kubectl
```
2. verify instration

```bash
turtlebot3@turtlebot3:~$ kubectl version --client
```

## setup telepresence

1. setup repository

```bash
turtlebot3@turtlebot3:~$ curl -s https://packagecloud.io/install/repositories/datawireio/telepresence/script.deb.sh | sudo bash
```
2. install telepresence

```bash
turtlebot3@turtlebot3:~$ sudo apt install --no-install-recommends telepresence -y
```

## start minikube without virtualization

### start minikube

1. create `.kube/config`

```bash
turtlebot3@turtlebot3:~$ rm -rf ~/.kube/
turtlebot3@turtlebot3:~$ mkdir -p $HOME/.kube
turtlebot3@turtlebot3:~$ touch $HOME/.kube/config
```
2. setup environment variables

```bash
turtlebot3@turtlebot3:~$ export MINIKUBE_HOME=$HOME
turtlebot3@turtlebot3:~$ export CHANGE_MINIKUBE_NONE_USER=true
turtlebot3@turtlebot3:~$ export KUBECONFIG=$HOME/.kube/config
```
3. start minikube without virtualization

```bash
turtlebot3@turtlebot3:~$ sudo -E minikube start --memory 2048 --vm-driver=none
```
4. verify minikube

```bash
turtlebot3@turtlebot3:~$ kubectl version
turtlebot3@turtlebot3:~$ kubectl get nodes
```

### confirm the dns settings of minkube

1. try to dig `www.google.com`

```bash
turtlebot3@turtlebot3:~$ kubectl run -it --rm --restart=Never dig --image tutum/dnsutils -- dig www.google.com
```
2. if you encountered `connection timed out`, you have to setup the additional nameservers to `kube-dns`

### setup additional nameservers to `kube-dns`

1. prepare `kube-dns-configmap.yaml` like below:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: kube-dns
  namespace: kube-system
  labels:
    addonmanager.kubernetes.io/mode: EnsureExists
data:
  upstreamNameservers: |-
    ["8.8.8.8", "8.8.4.4"]
```
2. apply `kube-dns-configmap.yaml`

```bash
turtlebot3@turtlebot3:~$ kubectl apply -f kube-dns-configmap.yaml
```
3. delete the pod of `kube-dns` (restart `kube-dns` automatically)

```bash
turtlebot3@turtlebot3:~$ kubectl delete pod -n kube-system $(kubectl get pods -n kube-system -l k8s-app=kube-dns -o template --template "{{(index .items 0).metadata.name}}")
```
4. retry to dig `www.google.com`

```bash
turtlebot3@turtlebot3:~$ kubectl run -it --rm --restart=Never dig --image tutum/dnsutils -- dig www.google.com
```

## create Secret to connect MQTT Broker

### prepare a command to create Secret of username & password

In [ ]:
echo "kubectl create secret generic mqtt-username-password --from-literal=mqtt_username=ros --from-literal=mqtt_password=${VERNEMQ__ros}"

1. create Secret of username & password using above command
2. create Secret of cert file

```bash
turtlebot3@turtlebot3:~$ kubectl create secret generic mqtt-ca-crt --from-file=/etc/mqtt-kube-operator/ca.crt
```

### start a pod to operate k8s Resources through MQTT

1. preapare `mqtt-kube-operator.yaml` like below:

```yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: mqtt-kube-operator
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: mqtt-kube-operator
  namespace: default
rules:
- apiGroups: [""]
  resources: ["services", "configmaps", "secrets"]
  verbs: ["get", "list", "create", "update", "delete"]
- apiGroups: ["apps"]
  resources: ["deployments"]
  verbs: ["get", "list", "create", "update", "delete"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: mqtt-kube-operator
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: mqtt-kube-operator
subjects:
- kind: ServiceAccount
  name: mqtt-kube-operator
  namespace: default
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mqtt-kube-operator
spec:
  replicas: 1
  selector:
    matchLabels:
      app: mqtt-kube-operator
  template:
    metadata:
      labels:
        app: mqtt-kube-operator
    spec:
      serviceAccountName: mqtt-kube-operator
      containers:
      - name: mqtt-kube-operator
        image: techsketch/mqtt-kube-operator:0.1.0
        imagePullPolicy: Always
        env:
        - name: MQTT_TLS_CA_PATH
          value: "/etc/mqtt/ca.crt"
        - name: MQTT_USERNAME
          valueFrom:
            secretKeyRef:
              name: mqtt-username-password
              key: mqtt_username
        - name: MQTT_PASSWORD
          valueFrom:
            secretKeyRef:
              name: mqtt-username-password
              key: mqtt_password
        - name: MQTT_HOST
          value: "<<your mqtt broker host>>"
        - name: MQTT_PORT
          value: "8883"
        - name: MQTT_APPLY_TOPIC
          value: "/mqtt-kube-operator/apply"
        - name: MQTT_DELETE_TOPIC
          value: "/mqtt-kube-operator/delete"
        volumeMounts:
        - mountPath: "/etc/mqtt"
          name: mqtt-ca-crt
          readOnly: true
      volumes:
      - name: mqtt-ca-crt
        secret:
          secretName: mqtt-ca-crt
```
2. apply `mqtt-kube-operator.yaml`

```bash
turtlebot3@turtlebot3:~$ kubectl apply -f mqtt-kube-operator.yaml
```
3. confirm that `mqtt-kube-operator` connect to MQTT Broker

```bash
turtlebot3@turtlebot3:~$ kubectl logs $(kubectl get pods -l app=mqtt-kube-operator -o template --template "{{(index .items 0).metadata.name}}")
```